In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['TIF']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [7]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 1]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [8]:
list(zip(longforms, counts))

[('tif', 85),
 ('tubulointerstitial fibrosis', 47),
 ('telomere dysfunction induced foci', 15),
 ('transoral incisionless fundoplication', 12),
 ('tumor interstitial fluid', 10),
 ('testicular interstitial fluid', 7),
 ('transcription initiation factor', 6),
 ('two independent experiments', 6),
 ('thalassaemia international federation', 4),
 ('trypanosomal immunosuppressive factor', 4),
 ('total impermeable film', 3),
 ('tetraiodofluorescein', 3),
 ('three independent experiments with similar results', 3),
 ('results is shown', 3),
 ('test information function', 2),
 ('tubular interstitial fibrosis', 2),
 ('tubule interstitial fibrosis', 2),
 ('triton insoluble f actin', 2),
 ('time to be inotrope free', 2),
 ('triplicate', 2),
 ('two independent experiments with similar results', 2)]

In [6]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-09-25 14:47:31] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.2/bio_ontology.pkl


In [7]:
result = [grounding_map, names, pos_labels]

In [8]:
result

[{'benzaldehyde lyase': 'MESH:C059416',
  'betaine aldehyde': 'CHEBI:CHEBI:15710',
  'bile salt activity lipase': 'HGNC:1848',
  'bioartificial liver': 'MESH:D019164',
  'blood alcohol levels': 'ungrounded',
  'breath alcohol levels': 'ungrounded',
  'british anti lewisite': 'CHEBI:CHEBI:64198',
  'brochoalveolar lavage': 'MESH:D018893',
  'bronchalveolar lavage': 'MESH:D018893',
  'bronchial alveolar lavage': 'MESH:D018893',
  'bronchial lavage': 'MESH:D018893',
  'bronchio alveolar lavage': 'MESH:D018893',
  'bronchiolar lavage': 'MESH:D018893',
  'broncho alveolar lavage': 'MESH:D018893',
  'bronchoalveolar': 'MESH:D018893',
  'bronchoalveolar fluid': 'MESH:D018893',
  'bronchoalveolar larvage': 'MESH:D018893',
  'bronchoalveolar lavage': 'MESH:D018893'},
 {'MESH:C059416': 'benzaldehyde lyase',
  'CHEBI:CHEBI:15710': 'betaine aldehyde',
  'HGNC:1848': 'CEL',
  'MESH:D019164': 'Liver, Artificial',
  'CHEBI:CHEBI:64198': 'dimercaprol',
  'MESH:D018893': 'Bronchoalveolar Lavage'},
 ['H

In [9]:
grounding_map, names, pos_labels = ({'telomere dysfunction induced foci': 'ungrounded',
  'test information function': 'ungrounded',
  'testicular interstitial fluid': 'ungrounded',
  'tetraiodofluorescein': 'CHEBI:CHEBI:61000',
  'thalassaemia international federation': 'ungrounded',
  'tissue interstitial fluid': 'ungrounded',
  'totally impermeable film': 'ungrounded',
  'transcription initiation factor': 'MESH:D035165',
  'transoral incisionless fundoplication': 'MESH:D018662',
  'trypanosomal immunosuppressive factor': 'ungrounded',
  'tubular interstitial fibrosis': 'MESH:D009395',
  'tubulo interstitial fibrosis': 'MESH:D009395',
  'tubulointerstitial fibrosis': 'MESH:D009395',
  'tumor interstitial fluid': 'ungrounded'},
 {'CHEBI:CHEBI:61000': 'erythrosin B',
  'MESH:D035165': 'Transcription Factors, General',
  'MESH:D018662': 'Fundoplication',
  'MESH:D009395': 'Nephritis, Interstitial'},
 ['CHEBI:CHEBI:61000', 'MESH:D009395', 'MESH:D018662', 'MESH:D035165'])

In [16]:
excluded_longforms = ['tif']

In [17]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [18]:
additional_entities = {}

In [19]:
unambiguous_agent_texts = {}

In [14]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [12]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [13]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [14]:
intersection1

[('HGNC:1848', 'HGNC:1848', 86)]

In [15]:
intersection2

[('MESH:D018893', 'HGNC:1848', 0),
 ('HGNC:1848', 'HGNC:1848', 1),
 ('ungrounded', 'HGNC:1848', 0),
 ('CHEBI:CHEBI:64198', 'HGNC:1848', 0),
 ('MESH:D019164', 'HGNC:1848', 0),
 ('MESH:C059416', 'HGNC:1848', 0),
 ('CHEBI:CHEBI:15710', 'HGNC:1848', 0)]

In [1]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

NameError: name 'unambiguous_agent_texts' is not defined

In [18]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [15]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-29 04:04:14] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-29 04:04:17] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9571717171717171 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [20]:
classifier.stats

{'label_distribution': {'MESH:D018893': 1259,
  'HGNC:1848': 86,
  'ungrounded': 17,
  'CHEBI:CHEBI:64198': 8,
  'MESH:D019164': 18,
  'MESH:C059416': 3,
  'CHEBI:CHEBI:15710': 2},
 'f1': {'mean': 0.993068, 'std': 0.002061},
 'precision': {'mean': 0.987107, 'std': 0.004015},
 'recall': {'mean': 0.999257, 'std': 0.001487},
 'CHEBI:CHEBI:15710': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'ungrounded': {'f1': {'mean': 0.67, 'std': 0.345832},
  'pr': {'mean': 0.616667, 'std': 0.331662},
  'rc': {'mean': 0.75, 'std': 0.387298}},
 'HGNC:1848': {'f1': {'mean': 0.978069, 'std': 0.031578},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.958889, 'std': 0.058542}},
 'MESH:D019164': {'f1': {'mean': 0.914286, 'std': 0.069985},
  'pr': {'mean': 0.85, 'std': 0.122474},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'MESH:D018893': {'f1': {'mean': 0.994077, 'std': 0.001757},
  'pr': {'mean': 0.999206, 'std': 0.001587},
  'rc': {'mean': 0.98900

In [20]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [21]:
disamb.dump(model_name, results_path)

In [22]:
print(disamb.info())

Disambiguation model for TIF

Produces the disambiguations:
	Fundoplication*	MESH:D018662
	Nephritis, Interstitial*	MESH:D009395
	Transcription Factors, General*	MESH:D035165
	erythrosin B*	CHEBI:CHEBI:61000

Class level metrics:
--------------------
Grounding                     	Count	F1     
       Nephritis, Interstitial*	39	0.96444
                    Ungrounded	30	   0.96
                Fundoplication*	 8	    1.0
Transcription Factors, General*	 5	    1.0
                  erythrosin B*	 2	    0.2

Weighted Metrics:
-----------------
	F1 score:	0.95717
	Precision:	0.93682
	Recall:		0.98182

* Positive labels
See Docstring for explanation



In [23]:
model_to_s3(disamb)